In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from transformers import PreTrainedModel, LlamaConfig, LlamaModel, LlamaTokenizer
import torch.nn as nn
import torch
from typing import Optional, List

# for i in range(torch.cuda.device_count()):
#    print(torch.cuda.get_device_properties(i).name)
# cuda = torch.device('cuda')
# b1 = torch.tensor([1., 2.]).to(device=cuda)

In [2]:
from transformers import PreTrainedModel, LlamaConfig, LlamaModel, LlamaTokenizer
import torch.nn as nn
import torch
from typing import Optional, List

class LlamaRewardModel(PreTrainedModel):
    config_class = LlamaConfig
    def __init__(self, config):
        super().__init__(config)
        self.model = LlamaModel(config)
        self.regression_head = nn.Linear(self.config.hidden_size, 1, bias=False)

    def forward( # args are the same as LlamaForCausalLM
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):

        transformer_outputs = self.model(
                                input_ids,
                                attention_mask=attention_mask,
                                position_ids=position_ids,
                                past_key_values=past_key_values,
                                inputs_embeds=inputs_embeds,                               
                            )

        hidden_states = transformer_outputs[0]
        rewards = self.regression_head(hidden_states).squeeze(-1)
        
        ends = attention_mask.cumsum(dim=1).argmax(dim=1).view(-1,1)
        rewards = torch.gather(rewards, 1, ends)
        
        return rewards

tokenizer = LlamaTokenizer.from_pretrained("openbmb/UltraRM-13b", cache_dir='/scratch/henrypapadatos')

#load the model in 16-bit precision
model = LlamaRewardModel.from_pretrained("openbmb/UltraRM-13b", cache_dir='/scratch/henrypapadatos')

model = model.half()

#put the model on the GPU in evaluation mode
if torch.cuda.is_available():
    device = torch.device("cuda")
    model = model.cuda()
else:
    device = torch.device("cpu")

model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
#print the memory footprint of the model
print(f"Model size: {sum(p.numel() for p in model.parameters())/1e6:.2f}Mb")
#print the device of the model
print(next(model.parameters()).device)

Model size: 12852.03Mb
cuda:0


In [11]:
ultrarm_template = """Human: {instruction}

Assistant: {completion}"""

dataset = [ # cases in webgpt
  {"chosen":"Human: \"Who orders martinis \"\"shaken, not stirred\"\"?\"\nAssistant: Sean Connery's character, fictional British Secret Service agent James Bond, in the movie Goldfinger, stated that he preferred his martini to be \"shaken, not stirred\". [1] Some believe that Bond ordered his martini shaken because of the vodka it contained, as vodka was, for the most part, refined from potatoes (cheaper brands) which made the vodka oily. To disperse the oil, Bond ordered his martinis shaken. [2]","rejected":"Human: \"Who orders martinis \"\"shaken, not stirred\"\"?\"\nAssistant: Fleming's fictional British Secret Service agent James Bond orders his martini cocktail shaken, not stirred [1]. Bond's preferences for his martini are carried over to the films, where his orders are seen in both the 1961 film Dr. No and the 2006 film Casino Royale [1, 2]. In both films, Bond's subordinates copy his order, telling the bartender to keep the fruit with their drinks [2]. However, in the 2006 film, Bond appears irritated when the bartender asks if he would like his drink shaken or stirred [2]."},
  {"chosen":"Human: Sural relates to which part of the body?\nAssistant: The sural region is the muscular swelling of the back of the leg below the knee, formed chiefly by the bellies of the gastrocnemius and soleus muscles [1,2].","rejected":"Human: Sural relates to which part of the body?\nAssistant: The Sural nerve runs down the side of the leg near the small saphenous vein, then passes forward below the lateral malleolus and continues on the outside of the foot as the lateral dorsal cutaneous nerve, which then communicates with the intermediate dorsal cutaneous nerve, which branches off to the side of the foot. [1]"}
]


for example in dataset:
    inputs = tokenizer(example["chosen"], return_tensors="pt").to(device)
    chosen_reward = model(**inputs).item()
    inputs = tokenizer(example["rejected"], return_tensors="pt").to(device)
    rejected_reward = model(**inputs).item()
    print(chosen_reward - rejected_reward)



2.418914794921875
0.2109375


**32 bits precision**

2.4158289432525635

0.18967914581298828

**16 bits precision**

2.418914794921875

0.2109375

In [9]:
test = [
'''Human: Hello, what is your name?
Assistant: Hello, I am a helpful assistant. What can I help you with today?''',
'''Human: Hello, what is your name?
Assistant: Fuck you!'''
]

for el in test: 
    inputs = tokenizer(el, return_tensors="pt").to(device)
    reward = model(**inputs).item()
    print(reward)

-7.74609375
-11.4609375


**32 bits precision**

-7.718698024749756

-11.430608749389648

**16 bits precision**

-7.74609375

-11.4609375